# Class 15: Pandas Basics
## Objective: Learn how to use Pandas to load, inspect, and manipulate structured datasets

Pandas is the industry-standard tool for data manipulation in Python. It was designed specifically to handle "labeled" data. This is very helpful for many applications in astronomy that require more than simple NumPy arrays.

This notebook is based on https://tingyuansen.github.io/coding_essential_for_astronomers/lectures/lecture11-pandas-basics.html

In [ ]:
import pandas as pd
print(f"Pandas version: {pd.__version__}")

import numpy as np
import matplotlib.pyplot as plt

# Set display options for better visibility
pd.set_option('display.max_rows', 100)      # Show up to 100 rows before truncating
pd.set_option('display.max_columns', 20)    # Show up to 20 columns before truncating
pd.set_option('display.width', 120)         # Wider display for more columns
pd.set_option('display.precision', 3)       # Show 3 decimal places
pd.set_option('display.float_format', '{:.3f}'.format)  # Consistent float formatting

## Section 1: Introduction to Pandas and Data Structures

Pandas is built on top of NumPy and introduces two primary data structures: the **Series** (1D) and the **DataFrame** (2D). Unlike NumPy arrays, these structures use labeled axes (indices), which allow you to access data using meaningful names (like "RA" or "Object Name") rather than just numerical positions.

In [ ]:
# A Series is like a single column of data with labels (index=[] provides the lables)
magnitudes = pd.Series([3.4, 6.1, 8.0], index=['M31', 'M42', 'M51'], name='Magnitude')

# A DataFrame is a 2D table (collection of Series)
data = {
    'Type': ['Spiral', 'Nebula', 'Spiral'],
    'Constellation': ['Andromeda', 'Orion', 'Canes Venatici']
}
df_messier = pd.DataFrame(data, index=['M31', 'M42', 'M51'])

print("Series with Messier number as index:")
print(magnitudes)
print("\nDataFrame Example with Messier number as index:")
print(df_messier)

**Test your understanding:** Create a Pandas Series named `object_types` that contains the types of three Messier objects of your choice. Use the Messier numbers (e.g., 'M1', 'M8', 'M45') as the index labels. These correspond to the Crab Nebula, Lagoon Nebula, and the Pleiades star cluster.

In [ ]:
# Starter code
# object_types = pd.Series([]) 

## Section 2: Data Input and Inspection
In astronomy, data are often stored in CSV (Comma Separated Values) files. Pandas makes it easy to load these using `pd.read_csv()`. Once loaded, you should always inspect your data using methods like `.head()` (viewing the top rows), `.info()` (checking data types and missing values), and `.describe()` (calculating summary statistics).

Grab the messier_objects.csv files from Carmen or the https://github.com/paulmartini/AstroDataAnalysis repository to run the code below.

In [ ]:
# Load the Messier objects dataset used in the lecture
df = pd.read_csv('../data/messier_objects.csv')

# Inspect the first 5 rows
print("First 5 rows:")
display(df.head())  # or use .tail() for the end

# Get a summary of the columns and data types
print("\nDataset Info:")
df.info()

# Get the shape
print(f"\nDataset shape: {df.shape} (rows, columns)") 

**Test your understanding:** Use a Pandas method to display the last 10 rows of the `df` DataFrame. Then, use an attribute to print the list of all column names in the dataset.

In [ ]:
# Display last 10 rows:

# Print column names:


## Section 3: Data Selection and Indexing

You can select specific columns using bracket notation (like a dictionary) or specific rows using labels (`.loc[]`) and integer positions (`.iloc[]`). Selecting data is the first step in narrowing down a large catalog to just the specific stars or galaxies you want to study.

There is an important difference between these two:
- `.iloc[]` is sliced like a list, so `.ilog[0:2]` returns 0 and 1
- `.loc[]` is includes both endpoints

In [ ]:
# Selecting a single column
mags = df['magnitude']
print(f"\nmags = ") 
display(mags)

# Selecting multiple columns
coordinates = df[['ra', 'dec']]
print(f"\ncoordinates = ") 
display(coordinates) 

# Selecting a specific row by integer index (the first row)
first_five = df.iloc[0:5]
print(f"\nFirst five rows: \n{first_five}")

# Selecting a specific slice of rows and columns
subset = df.loc[0:5, ['name', 'magnitude']]
print(f"\nSpecific subset from 0 through 5 subset (inclusive): \n{subset}")

**Test your understanding:** Select only the name and constellation columns for the objects located at integer indices 20 through 25 (inclusive). Store this in a variable called my_selection.

In [ ]:
# Enter your code here
# my_selection = 

## Section 4: Filtering and Boolean Indexing
Filtering allows you to extract subsets of data that meet specific astronomical criteria. You can use boolean conditions (e.g., `df['magnitude'] < 6`) and combine them using `&` (and) or `|` (or). This is essential for tasks like selecting "naked-eye" objects or objects within a specific constellation.

In [ ]:
# Filter for objects brighter than magnitude 6 (lower number = brighter)
bright_objects = df[df['magnitude'] < 6.0]

# Complex filter: Bright objects in the constellation 'Orion'
orion_bright = df[(df['magnitude'] < 7.0) & (df['constellation'] == 'Orion')]

print(f"Found {len(orion_bright)} bright objects in Orion.")

**Test your understanding:** Create a new DataFrame called **nebulae** that contains only the objects where the type column is exactly `'Nebula'` AND the `magnitude` is greater than (fainter than) `8.0`.

In [ ]:
# Enter your code here
# nebulae = 
# display(nebulae)

## Section 5: Data Transformation and Manipulation
Pandas supports "vectorized" operations, meaning you can perform math on an entire column at once without writing a `for` loop. This is useful for unit conversions or calculating new properties, which you can then save as a new column in your DataFrame.

We'll conduct the next exercises with a small version (5000 entries) of the roughly two billion stars in the GAIA DR3 catalog. (More is available from https://www.cosmos.esa.int/web/gaia/dr3) 

### Load and explore GAIA DR3 data

Some notes on units:
- `parallax` is in units of milliarcseconds (or mas) 
- `pmra` and `pmdec` are in units of mas/yr
- `radial_velocity` is in units of km/s

In [ ]:
gaia = pd.read_csv('../data/data_gaia_dr3.csv')

print(f"Loaded {len(gaia)} stars from GAIA catalog")
print(f"DataFrame shape: {gaia.shape} (rows, columns)")

print(f"\nGAIA info:") 
gaia.info() 

print(f"\nGAIA catalog head:")
gaia.head(5)

In [ ]:
# Plot the sample on the sky

# Create all-sky map using Mollweide projection
fig = plt.figure(figsize=(12, 6))
ax = plt.subplot(111, projection='mollweide')

# Convert RA to radians and shift to [-pi, pi] range
# Mollweide projection expects longitude from -π to π
ra_rad = np.radians(gaia['ra'] - 180)
dec_rad = np.radians(gaia['dec'])

# Plot with color representing magnitude (brightness)
scatter = ax.scatter(ra_rad, dec_rad, 
                    c=gaia['phot_g_mean_mag'],  # Color by magnitude
                    s=1,                          # Small point size
                    cmap='viridis_r',            # Reversed viridis colormap
                    vmin=0, vmax=15)             # Magnitude range

ax.set_xlabel('Right Ascension')
ax.set_ylabel('Declination')
ax.grid(True, alpha=0.3)
plt.colorbar(scatter, label='G Magnitude (smaller = brighter)')
plt.title('Our Gaia Sample: 5000 Nearby Stars')
plt.tight_layout()
plt.show()

### Vectorized operations:

In [ ]:
# Calculate distance from parallax
gaia['distance_pc'] = 1000./(gaia['parallax'])

# Convert radial velocity from km/s to m/s
gaia['rv_ms'] = 1000.*gaia['radial_velocity']

**Test your understanding:** Create a new column in `gaia` called `distance_lyr` based on the column `distance_pc` you just computed in the previous cell. There are about 3.2616 light years in a parsec. 

In [ ]:
# Enter your code here
# (assuming 'distance_pc' exists)
# gaia['distance_lyr'] = ...

## Section 6: Handling Missing and Duplicated Data
Real-world astronomical catalogs often have missing data points, represented in Pandas as NaN (Not a Number). You must decide whether to remove these rows using `.dropna()` or fill them with a placeholder value using `.fillna()` to prevent errors in your calculations.

Note that if you drop NaN values and then want to add other columns to the DataFrame, you should use `.copy()` to make a "Deep Copy", that is you make a completely new DataFrame. Otherwise, you will be working with a **slice** of the original DataFrame.

In [ ]:
print(f"Length before cleaning is {len(gaia)}")

# Check how many missing values are in each column
print("Missing values per column:")
print(gaia.isnull().sum())

# Drop any rows that have a missing 'phot_bp_mean_mag'
gaia_cleaned = gaia.dropna(subset=['phot_bp_mean_mag']).copy()  # Create a new DataFrame
print(f"Length after cleaning is {len(gaia_cleaned)}")

# Look for duplicates:
duplicates = gaia['source_id'].duplicated().sum()
print(f"\nDuplicate source_ids: {duplicates}")

**Test your understanding:** 

Complete kinematic information for stars refers to their proper motions (columns `pmra` and `pmdec`) and radial velocity (`radial_velocity`). Identify how many stars have complete kinematic informations by defining a new parameter called `has_kinematics`: 

In [ ]:
# Enter your code here
# has_kinematics = ...

print(f"Stars with complete 3D velocities: {len(has_kinematics)}")
print(f"That's {100 * len(has_kinematics) / len(gaia):.1f}% of our sample")
print(f"\nLost {len(gaia) - len(has_kinematics)} stars due to missing radial velocities")

## Bonus Exercise: Search for high-velocity stars

Proper motion is measured in units of milli-arcseconds per year, as proper motion is measured in angular units (change in position in some time). 

Radial velocity is measured in km/s, that is velocity along the line of sight. This is perpendicular to proper motion. 

We need to convert proper motion into km/s to compute the total (3D) velocity of each star. This conversion requires the distance. The formula is:

$$ v_{tan} = 4.74 * \mu * d $$

where $v_{tan}$ is the velocity in the plane of the sky in km/s, $\mu$ is the proper motion in the plane of the sky in mas/yr, and $d$ is the distance in parsecs. 

In [ ]:
# Calculate total space velocity from proper motions and radial velocity
# Note that all of these calculations are vectorized

# Step 1: Total proper motion (combining RA and Dec components) and convert to as/yr
has_kinematics['pm_total_as_yr'] = 0.001 * np.sqrt(
    has_kinematics['pmra']**2 + has_kinematics['pmdec']**2
)

# Step 2: Convert proper motion to tangential velocity
# v_tangential (km/s) = 4.74 × μ (as/yr) × d (pc)
# The factor 4.74 comes from unit conversions
has_kinematics['v_tan'] = 4.74 * has_kinematics['pm_total_as_yr'] * has_kinematics['distance_pc']

# Step 3: Combine tangential and radial velocities
# Pythagorean theorem in 3D: v_total = √(v_tan² + v_radial²)
has_kinematics['v_total'] = np.sqrt(
    has_kinematics['v_tan']**2 + has_kinematics['radial_velocity']**2
)

print("Velocity calculation complete!")
print("\nVelocity statistics:")
print(has_kinematics['v_total'].describe())

# Plot the velocity histogram
plt.hist(has_kinematics['v_total'], bins=25, range=(0,250)) 
plt.xlabel(r"$v_{total}$ [km/s]")
plt.ylabel("Number of stars") 

## Bonus Exercise: Plot the HR diagram

The Hertzsprung-Russell (HR) diagram is one of the most famous and fundamental diagrams in astrophysics. TIt illustrates the relation between luminosity and temperature for stars. 

Stars spend the majority of their lifetimes on the main sequence of the HR diagram, which is when they are powered soley by the fusion of Hydrogen into Helium in their cores. These stars form a sequence on this diagram because more massive stars have higher surface temperatures. 

Once stars leave the main sequence, stars expand and their surface temperatures cool. This leads most of them to become red giants. Most of these stars then go through a second phase of expansion and become asymptotic giant branch stars. 

Here is how to convert the data from the GAIA catalog into a HR diagram.

In [ ]:
# Calculate absolute G-band magnitude
# Another vectorized operation across all stars
gaia_cleaned['abs_g_mean_mag'] = gaia_cleaned['phot_g_mean_mag'] - 5 * np.log10(gaia_cleaned['distance_pc']/10)

gaia_cleaned['Bp-Rp'] = gaia_cleaned['phot_bp_mean_mag'] - gaia_cleaned['phot_rp_mean_mag'] 

fig, ax = plt.subplots(1,1,figsize=(6,6))

ax.scatter(gaia_cleaned['Bp-Rp'], gaia_cleaned['abs_g_mean_mag'], s=10, label="GAIA DR3") 
ax.set_ylim(14, -2)
ax.set_xlabel(r'$Bp - Rp$', fontsize=14)
ax.set_ylabel(r'$M_g$', fontsize=14)
ax.legend(fontsize=14)